# Introduction to SPARQL, RDF, and LOD

While many databases, services, or museums might expose their data via a web API, there can be limitations. Matthew Lincoln has an excellent tutorial at [The Programming Historian](https://programminghistorian.org/en/lessons/graph-databases-and-SPARQL) that walks us through some of these differences, but the key one is in the way the data is represented. When data is described using a 'Resource Description Framework', RDF, the resource - the 'thing'- is described via a series of relationships, rather than as rows in a table or keys having values. 

Information is in the relationships. It's a network. It's a _graph_. Thus, every 'thing' in this graph can have its own _uniform resource identifier_ (URI) that lives as a location on the internet. Information can then be created by making _statements_ that use these URIs, similarly to how English grammar creates meaning: subject verb object. Or, in RDF-speak, 'subject predicate object', also known as a _triple_. In this way, data in _different_ places can be linked together by referencing the elements they have in common. This is Linked Open Data (LOD). The access point for interrogating LOD is called an 'endpoint'. 

Finally, _SPARQL_ is an acronymn for SPARQL Protocol and RDF Query Language (yes, it's one of _those_ kinds of acronyms).

In this notebook, we're not using Python or R directly. Instead, we've set up a 'kernel' (think of that as the 'engine' for the notebook) that already includes everything necessary to set up and run SPARQL queries. (For reference, the kernel code is [here](https://github.com/paulovn/sparql-kernel)). Both R and Python can interact with and query endpoints, and manipulate linked open data, but for the sake of learning a bit of what one can do with SPARQL, this notebook keeps all of that ancillary code tucked away. The [followup notebook](Using R to Retrieve and Visualize Data from SPARQL.ipynb) to this one shows you how to use R to do some basic manipulations of the query results.

---

## Simple RDF example

Here, we are following [Matthew Lincoln's tutorial](https://programminghistorian.org/en/lessons/graph-databases-and-SPARQL).  

Let's look at his example, which concerns the painting, 'The Nightwatch'. 

`<The Nightwatch> <was created by> <Rembrandt van Rijn> .`

This _statement_ has three elements: 

+ the subject: `<The Nightwatch>`
+ the predicate: `<was created by>`
+ the object: `<Rembrandt van Rijn>`

Lincoln combines these, and other such statements, into a (pseudo-)RDF database like so:

```
<The Nightwatch> <was created by> <Rembrandt van Rijn> .
<The Nightwatch> <was created in> <1642> .
<The Nightwatch> <has medium> <oil on canvas> .
<Rembrandt van Rijn> <was born in> <1606> .
<Rembrandt van Rijn> <has nationality> <Dutch> .
<Johannes Vermeer> <has nationality> <Dutch> .
<Woman with a Balance> <was created by> <Johannes Vermeer> .
<Woman with a Balance> <has medium> <oil on canvas> .
```

Such RDF databases are describing nodes and links, and so we can visualize as a graph like so:

![A network visualization of the pseudo-RDF shown above. Arrows indicate the ‘direction’ of the predicate. For example, that ‘Woman with a Balance was created by Vermeer’, and not the other way around.](https://programminghistorian.org/images/graph-databases-and-SPARQL/sparql01.svg)

But there is a difference between the pseudo-RDF that Lincoln shows us, and what _actual_ RDF might look like:

```
<http://data.rijksmuseum.nl/item/8909812347> <http://purl.org/dc/terms/creator>  <http://dbpedia.org/resource/Rembrandt>
```

The human-readable version requires _more_ statements:

```
<http://data.rijksmuseum.nl/item/8909812347> <http://purl.org/dc/terms/title> "The Nightwatch" .

<http://purl.org/dc/terms/creator> <http://www.w3.org/1999/02/22-rdf-syntax-ns#label> "was created by" .

<http://dbpedia.org/resource/Rembrandt> <http://xmlns.com/foaf/0.1/name> "Rembrandt van Rijn" .
```

This is just a quick introduction; please do examine [Lincoln's tutorial](https://programminghistorian.org/en/lessons/graph-databases-and-SPARQL) for more details. But now, let's explore how this notebook can be used to write some queries.



In [ ]:
# Jupyter notebooks have various built-in commands called 'magics' that are accessed with the '%' character; these depend on the kernel. 
# Let's see what the SPARQL kernel has
%lsmagics

In [ ]:
# when using this notebook, the first thing we have to do - or rather, the first time we run _any_ query,
# is to tell it what endpoint we're going to use. Let's use the British Museum's:

%endpoint http://edan.si.edu/saam/sparql

Lincoln suggests that when we first encountered a new RDF graph, that we explore the network of relationships from an example object to understand what is going on in the database, to see what is available for querying. Since we're querying the British Museum, let's take [the Rosetta Stone](http://collection.britishmuseum.org/id/object/YCA62958) as our example.

In the query below, `p` and `o` stand for 'predicate' and 'object'. Thus, we're building up a query that asks, 'show me every statment structured `<The Rosetta Stone> <predicate> <object>`. When the results load up, you can right-click on each statement (which is a URI, remember) to see what we've discovered. This could give you the necessary information to construct more complicated queries.

*Nb* The British Museum sparql endpoint and the underlying infrastructure does not appear to be well supported. Results are sometimes flaky or not reachable.

In [ ]:
SELECT ?p ?o
WHERE {
  <http://edan.si.edu/saam/id/object/1980.71> ?p ?o .
}


In this next query, we look for objects in the collection that have the label 'fibula'.

In [ ]:
%endpoint http://edan.si.edu/saam/sparql
%display table
PREFIX bmo: <http://www.researchspace.org/ontology/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

SELECT ?object
WHERE {

  # Search for all values of ?object that have a given "object type"
  ?object bmo:PX_object_type ?object_type .

  # That object type should have the label "fibula"
  ?object_type skos:prefLabel "fibula" .
}
LIMIT 10

## Wikidata

Wikidata is another endpoint we can query. Below we have a query by Sebastian Heath that extracts some of the genealogical data on Roman emperors contained in that database. The `wd:Q842606` can be expanded to refer to [https://www.wikidata.org/wiki/Q842606](https://www.wikidata.org/wiki/Q842606), which describes the concept 'Roman Emperor'. `wdt:P39` is a predicate meaning 'Position held' [https://www.wikidata.org/wiki/Property:P39](https://www.wikidata.org/wiki/Property:P39). 

In [ ]:
%endpoint http://query.wikidata.org/sparql
%display table

SELECT ?emperorLabel ?emperor_dob
       ?childLabel
       ?motherLabel ?maternalGrandfatherLabel ?maternalGrandmotherLabel
       ?emperor ?child ?mother ?maternalGrandfather ?maternalGrandmother WHERE {
  
  ?emperor wdt:P39 wd:Q842606 . #p39: position held. Q842606: Roman Emperor
  ?emperor wdt:P569 ?emperor_dob . # p569: date of birth
  ?child wdt:P22 ?emperor . #p22: father
  ?child wdt:P25 ?mother .  #p25: mother
  OPTIONAL { ?mother wdt:P22 ?maternalGrandfather }
  OPTIONAL { ?mother wdt:P25 ?maternalGrandmother }
  
 # automatic label expander
 SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
 
} ORDER BY ?emperor_dob


Let's visualize these relationships. We're running the same query, but we use CONSTRUCT to create the nodes and edges that represent these familial relationships. We want to show 'emperor x is the father of person y' and 'person a is the mother of person y'.  That gives us the structure. To get the content, we run the SELECT command where we first tell it to retrieve those individuals who were emperor, and then retrieve the children data.  

Once you've run the query, use ctrl+f to find someone familiar, like Augustus (Q1405). In the resulting graph, an edge labeled 'p22' eg Q1405 ->P22 -> Q2259 can be read, 'Q1405 is the father of Q2259', or rather, 'Augustus is the father of Julia the Elder'.

Roman geneaology.... it's complicated! 

In [ ]:
%endpoint http://query.wikidata.org/sparql
%display diagram 

CONSTRUCT {
         ?emperor wdt:P22 ?child . #p22: father
         ?mother wdt:P25 ?child .  #p25: mother 
          }

WHERE {
  
  ?emperor wdt:P39 wd:Q842606 .
  ?child wdt:P22 ?emperor . #p22: father
  ?child wdt:P25 ?mother .  #p25: mother
  OPTIONAL { ?mother wdt:P22 ?maternalGrandfather }
  OPTIONAL { ?mother wdt:P25 ?maternalGrandmother }
} 

## Nomisma

Another excellent SPARQL endpoint is the Nomisma portal for numismatic materials. 

http://nomisma.org/sparql

In [ ]:
%endpoint http://nomisma.org/query

Now, if you actually go to [http://query.wikidata.org/sparql](http://query.wikidata.org/sparql) you'll find a query builder with the following information already preloaded:

```
PREFIX rdf:	<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX bio:	<http://purl.org/vocab/bio/0.1/>
PREFIX crm:	<http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype:	<http://purl.org/dc/dcmitype/>
PREFIX dcterms:	<http://purl.org/dc/terms/>
PREFIX foaf:	<http://xmlns.com/foaf/0.1/>
PREFIX geo:	<http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm:	<http://nomisma.org/id/>
PREFIX nmo:	<http://nomisma.org/ontology#>
PREFIX org:	<http://www.w3.org/ns/org#>
PREFIX osgeo:	<http://data.ordnancesurvey.co.uk/ontology/geometry/>
PREFIX rdac:	<http://www.rdaregistry.info/Elements/c/>
PREFIX skos:	<http://www.w3.org/2004/02/skos/core#>
PREFIX spatial: <http://jena.apache.org/spatial#>
PREFIX void:	<http://rdfs.org/ns/void#>
PREFIX xsd:	<http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
  ?s ?p ?o
} LIMIT 100
```

All those prefixes are the ontologies being used to describe the materials. The `?s ?p ?o` are the subject, predicate, objects that we're going to search for. Let's run some of the [example queries](http://nomisma.org/documentation/sparql) that Nomisma can handle. Since Roman Emperors are often depicted on coins, let's see which emperors are present in Nomisma.

In [ ]:
%display table 
PREFIX rdf:	<http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX bio:	<http://purl.org/vocab/bio/0.1/>
PREFIX crm:	<http://www.cidoc-crm.org/cidoc-crm/>
PREFIX dcmitype:	<http://purl.org/dc/dcmitype/>
PREFIX dcterms:	<http://purl.org/dc/terms/>
PREFIX foaf:	<http://xmlns.com/foaf/0.1/>
PREFIX geo:	<http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nm:	<http://nomisma.org/id/>
PREFIX nmo:	<http://nomisma.org/ontology#>
PREFIX org:	<http://www.w3.org/ns/org#>
PREFIX osgeo:	<http://data.ordnancesurvey.co.uk/ontology/geometry/>
PREFIX rdac:	<http://www.rdaregistry.info/Elements/c/>
PREFIX skos:	<http://www.w3.org/2004/02/skos/core#>
PREFIX spatial: <http://jena.apache.org/spatial#>
PREFIX void:	<http://rdfs.org/ns/void#>
PREFIX xsd:	<http://www.w3.org/2001/XMLSchema#>

SELECT ?uri ?label WHERE {
?uri a foaf:Person ;
  skos:prefLabel ?label ;         
  org:hasMembership ?membership .
?membership org:role nm:roman_emperor .
FILTER(langMatches(lang(?label), "EN"))
} 

We can also do spatial queries; this one looks coins from mints around Athens.

It also specifies the format in which we wants the results returned, and to write these results to a json file for further manipulation.

In [ ]:
%format json  
%display table 
%outfile ../data/mints.json 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX geo: <http://www.w3.org/2003/01/geo/wgs84_pos#>
PREFIX nmo: <http://nomisma.org/ontology#>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT * WHERE {
   ?loc geo:lat ?lat ;
        geo:long ?long .

   ?mint geo:location ?loc ;
         skos:prefLabel ?label ;
         a nmo:Mint .

   FILTER langMatches(lang(?label), "en")

   FILTER (
      ?lat > 37.5 && ?lat < 38.4 &&
      ?long > 23.0 && ?long < 24.5
   )
}
